In [1]:
%matplotlib inline

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB :
  
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/Github

  repository= 'keyword_spotting'
  %cd {repository}
  %ls -a
  # CPU:
  #!pip install pydub torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

  # GPU:
  !pip install pydub torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 -f https://download.pytorch.org/whl/torch_stable.html
    
working_dir=%pwd
if 'PROJET_ESSAI/notebooks' in working_dir :
    print('Wrong working directory')
    %cd ..
    working_dir=%pwd
    print('New directory :',working_dir)

if 'storage' not in locals():   
    storage=dict()

Wrong working directory
/Users/carlos/Documents/LeTaffe/PROJET_ESSAI
New directory : /Users/carlos/Documents/LeTaffe/PROJET_ESSAI


In [ ]:
args.no_validation

In [ ]:
%run main.py --num-epochs 1

In [2]:
%run main.py --num-epochs 1

imports done
parsing variables...
Using : cpu
Training mode
training loader set up
validation loader set up
training/validation sets finished in  0:01:23.461948
1  models have been set up


  0%|          | 0.001053740779768177/1 [00:23<6:03:35, 21838.45s/it]

Train Epoch: 1 [0/84843 (0%)]	Loss: 4.029365


  2%|▏         | 0.022128556375131718/1 [07:41<5:37:56, 20734.99s/it] 

Train Epoch: 1 [2000/84843 (2%)]	Loss: 3.547853


...

 11%|█         | 0.10642781875658595/1 [36:25<5:00:53, 20204.16s/it]

Train Epoch: 1 [10000/84843 (12%)]	Loss: 3.472052


 11%|█         | 0.10958904109589049/1 [37:34<5:05:16, 20570.74s/it]


training program finished in  0:37:34.342032


KeyboardInterrupt: 

In [ ]:
storage['device'] = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using',storage['device'])

PATH='spectr.pt'
storage['training_mode']=True

## Downloading and setting up the dataset

In [ ]:
with timeThat('training & test sets'):
    root='/content/sample_data' if IN_COLAB else './'
    if 'train_set' not in locals():  
        train_set = SubsetSC("training",root)
    if 'test_set' not in locals():
        test_set = SubsetSC("testing",root)
storage['waveform'], storage['sample_rate'], label, speaker_id, utterance_number = train_set[0]

# TODO : We are doing it without resampling it.
#resample_transform=torchaudio.transforms.Resample(orig_freq=storage['sample_rate'], new_freq=new_sample_rate)

In [ ]:
print(len(train_set))
print(len(test_set))

## Let's visualize the data !

In [ ]:
plt.plot(storage['waveform'].t().numpy())

In [ ]:
from scipy import signal
from scipy.fft import fftshift

fe=16000
noverlap=int(20e-3*fe)
nperseg=int(30e-3*fe)
f, t, Sxx = signal.spectrogram(storage['waveform'][0], fe,noverlap=noverlap,nperseg=nperseg)
plt.pcolormesh(t, f, Sxx)
print(Sxx.shape)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [ ]:

n_mels=50
win_length=int(30e-3 *fe)
hop_length=int(10e-3 *fe)
n_fft=win_length 
spec_size=(fe -win_length)/hop_length +1
print('size of tensor should be ',spec_size)

### Spectrogram transform

In [ ]:
spectrogram_transform = torchaudio.transforms.Spectrogram(n_fft=n_fft,win_length=win_length,hop_length=hop_length)

spectrogram=spectrogram_transform(storage['waveform'])
print("Shape of spectrogram: {}".format(spectrogram.size()))

plt.figure()
plt.imshow(spectrogram.log2()[0].numpy())

## Resampling the data and setting the transform

In [ ]:
layers =[]
layers.append(spectrogram_transform)

storage['transform'] = nn.Sequential(*layers)

waveform_size=storage['transform'](storage['waveform']).shape
print(waveform_size)

## Setting up the test and train loader

In [ ]:
batch_size = 100

if storage['device'] == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False
    
storage['train_loader']  = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=train_set.collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)
storage['test_loader']  = torch.utils.data.DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=test_set.collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)

## Define the network

In [ ]:
storage['model'] = spectrogram_model(input_shape=waveform_size, n_output=len(train_set.labels))
if not storage['training_mode'] :
    storage['model'].load_state_dict(torch.load(PATH))
    storage['model'].eval()
storage['model'].to(storage['device'])
#print(storage['model'])
print("Number of parameters: %s" % storage['model'].count_parameters())

## Define the optimizer, loss function & metrics

In [ ]:
# Define the Optimizer
lr=0.001 
storage['optimizer'] = optim.Adam(storage['model'].parameters(), lr=lr, weight_decay=0.0001)
storage['scheduler'] = optim.lr_scheduler.StepLR(storage['optimizer'], step_size=20,
                                      gamma=0.1)  # reduce the learning after 20 epochs by a factor of 10
#Define the loss Function
storage['lossFunc']=F.cross_entropy
#Define the metrics :
storage['metrics']=metrics.countCorrectOutput

## Define the log interval and epochs

In [ ]:
storage['log_interval'] = 20
storage['n_epoch'] = 2

storage['pbar_update']  = 1 / (len(storage['train_loader']) + len(storage['test_loader']))
storage['losses'] = []
# The transform needs to live on the same device as the model and the data.

storage['transform'] = storage['transform'].to(storage['device'])

## Training

In [ ]:
if storage['training_mode'] :
    with timeThat('Main program') :
        with tqdm(total=storage['n_epoch']) as pbar:
            storage['pbar']=pbar
            for epoch in range(1, storage['n_epoch'] + 1):
                storage['epoch']=epoch
                train(storage)
                test(storage)
                storage['scheduler'].step()

In [ ]:
plt.plot(storage['losses']);
plt.title("training loss");

In [ ]:
FirstLayerWeights = storage['model'].conv1.weight.detach().cpu().numpy()

In [ ]:
plot_kernels2D(FirstLayerWeights)

In [ ]:
a=FirstLayerWeights[0,0].T
plt.imshow(a)
plt.show()

In [ ]:
torch.save(storage['model'].state_dict(), PATH)